## HANA FINTECH LAB - DATA EXTRACTING
1. RETREIVE WEEKLY DATA 
 - (where) exclude : negative payment 
 - revisit_his : addr, card_name, cnt_pay
 - inner join : company_id(co_name, cate, cate1), epopcon_id(latitude1, longitude1)
 * cardname, co_name, addr - refining required
2. MERGE & SUM+GROUP BY 
3. EXPORT AS XML FILE


In [1]:
import pymysql, pandas as pd

### REVISIT HIS DATA

In [5]:
conn = pymysql.connect(host='133.186.146.142', user='eums', password='eums00!q', db='eums-business', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute('select EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT from MEUMS_COMP_REVISIT_HIS where PAYMENT > 0 AND DEAL_DT BETWEEN "2018-03-01 00:00:00" AND "2018-03-06 23:59:59";')
p1 = pd.DataFrame( curs.fetchall() )
p1.to_pickle('revisit_his_p1.pick')

curs.execute('select EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT from MEUMS_COMP_REVISIT_HIS where PAYMENT > 0 AND DEAL_DT BETWEEN "2018-03-07 00:00:00" AND "2018-03-12 23:59:59";')
p2 = pd.DataFrame( curs.fetchall() )
p2.to_pickle('revisit_his_p2.pick')

curs.execute('select EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT from MEUMS_COMP_REVISIT_HIS where PAYMENT > 0 AND DEAL_DT BETWEEN "2018-03-13 00:00:00" AND "2018-03-18 23:59:59";')
p3 = pd.DataFrame( curs.fetchall() )
p3.to_pickle('revisit_his_p3.pick')

curs.execute('select EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT from MEUMS_COMP_REVISIT_HIS where PAYMENT > 0 AND DEAL_DT BETWEEN "2018-03-19 00:00:00" AND "2018-03-24 23:59:59";')
p4 = pd.DataFrame( curs.fetchall() )
p4.to_pickle('revisit_his_p4.pick')

curs.execute('select EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT from MEUMS_COMP_REVISIT_HIS where PAYMENT > 0 AND DEAL_DT BETWEEN "2018-03-25 00:00:00" AND "2018-03-31 23:59:59";')
p5 = pd.DataFrame( curs.fetchall() )
p5.to_pickle('revisit_his_p5.pick')
conn.close()

print(p1.shape) # 
print(p2.shape)
print(p3.shape)
print(p4.shape)
print(p5.shape)


(3500612, 4)
(3558165, 4)
(3491741, 4)
(4173992, 4)


### LIFEZONE

In [19]:
conn = pymysql.connect(host='133.186.146.142', user='eums', password='eums00!q', db='eums-business', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute('select EPOPCON_ID, LATITUDE1, LONGITUDE1 from MEUMS_LATEST_LIFEZONE WHERE EPOPCON_ID IN '+str(tuple(p1.EPOPCON_ID.unique().tolist()))+';') # array > list > tuple
p1_lifezone = pd.DataFrame( curs.fetchall() )
curs.execute('select EPOPCON_ID, LATITUDE1, LONGITUDE1 from MEUMS_LATEST_LIFEZONE WHERE EPOPCON_ID IN '+str(tuple(p2.EPOPCON_ID.unique().tolist()))+';') # array > list > tuple
p2_lifezone = pd.DataFrame( curs.fetchall() )
curs.execute('select EPOPCON_ID, LATITUDE1, LONGITUDE1 from MEUMS_LATEST_LIFEZONE WHERE EPOPCON_ID IN '+str(tuple(p3.EPOPCON_ID.unique().tolist()))+';') # array > list > tuple
p3_lifezone = pd.DataFrame( curs.fetchall() )
curs.execute('select EPOPCON_ID, LATITUDE1, LONGITUDE1 from MEUMS_LATEST_LIFEZONE WHERE EPOPCON_ID IN '+str(tuple(p4.EPOPCON_ID.unique().tolist()))+';') # array > list > tuple
p4_lifezone = pd.DataFrame( curs.fetchall() )
curs.execute('select EPOPCON_ID, LATITUDE1, LONGITUDE1 from MEUMS_LATEST_LIFEZONE WHERE EPOPCON_ID IN '+str(tuple(p5.EPOPCON_ID.unique().tolist()))+';') # array > list > tuple
p5_lifezone = pd.DataFrame( curs.fetchall() )
conn.close()

# p1 = pd.merge(p1, p1_lifezone, how='inner', on='EPOPCON_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','LATITUDE1','LONGITUDE1']]
# p2 = pd.merge(p2, p2_lifezone, how='inner', on='EPOPCON_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','LATITUDE1','LONGITUDE1']]
# p3 = pd.merge(p3, p3_lifezone, how='inner', on='EPOPCON_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','LATITUDE1','LONGITUDE1']]
# p4 = pd.merge(p4, p4_lifezone, how='inner', on='EPOPCON_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','LATITUDE1','LONGITUDE1']]
# p5 = pd.merge(p5, p5_lifezone, how='inner', on='EPOPCON_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','LATITUDE1','LONGITUDE1']]

lifezone = pd.DataFrame([])
lifezone = lifezone.append(p1_lifezone)
lifezone = lifezone.append(p2_lifezone)
lifezone = lifezone.append(p3_lifezone)
lifezone = lifezone.append(p4_lifezone)
lifezone = lifezone.append(p5_lifezone)
lifezone = lifezone.drop_duplicates('EPOPCON_ID')

lifezone.to_pickle('lifezone.pick')
print(lifezone.shape)

(547562, 3)


### COMPANY INFO 

In [17]:
# conn = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-cj-business', charset='utf8')
conn = pymysql.connect(host='133.186.143.151', user='eumsread', password='eumsread00!q', db='eums-shared', port=13306,charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute('select ID AS COMPANY_ID, CO_NAME, CATE,CATE1,ADDR from MEUMS_COMPANY WHERE ID IN '+str(tuple(p1.COMPANY_ID.unique().tolist()))+';') # array > list > tuple
p1_company = pd.DataFrame( curs.fetchall() )
curs.execute('select ID AS COMPANY_ID, CO_NAME, CATE,CATE1,ADDR from MEUMS_COMPANY WHERE ID IN '+str(tuple(p2.COMPANY_ID.unique().tolist()))+';') # array > list > tuple
p2_company = pd.DataFrame( curs.fetchall() )
curs.execute('select ID AS COMPANY_ID, CO_NAME, CATE,CATE1,ADDR from MEUMS_COMPANY WHERE ID IN '+str(tuple(p3.COMPANY_ID.unique().tolist()))+';') # array > list > tuple
p3_company = pd.DataFrame( curs.fetchall() )
curs.execute('select ID AS COMPANY_ID, CO_NAME, CATE,CATE1,ADDR from MEUMS_COMPANY WHERE ID IN '+str(tuple(p4.COMPANY_ID.unique().tolist()))+';') # array > list > tuple
p4_company = pd.DataFrame( curs.fetchall() )
curs.execute('select ID AS COMPANY_ID, CO_NAME, CATE,CATE1,ADDR from MEUMS_COMPANY WHERE ID IN '+str(tuple(p5.COMPANY_ID.unique().tolist()))+';') # array > list > tuple
p5_company = pd.DataFrame( curs.fetchall() )
conn.close()

# p1 = pd.merge(p1, p1_company, how='inner', on='COMPANY_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','CO_NAME', 'CATE','CATE1','ADDR' ]]
# p2 = pd.merge(p2, p2_company, how='inner', on='COMPANY_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','CO_NAME', 'CATE','CATE1','ADDR' ]]
# p3 = pd.merge(p3, p3_company, how='inner', on='COMPANY_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','CO_NAME', 'CATE','CATE1','ADDR' ]]
# p4 = pd.merge(p4, p4_company, how='inner', on='COMPANY_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','CO_NAME', 'CATE','CATE1','ADDR' ]]
# p5 = pd.merge(p5, p5_company, how='inner', on='COMPANY_ID')[['EPOPCON_ID','COMPANY_ID','CARD_NAME''PAYMENT','CO_NAME', 'CATE','CATE1','ADDR' ]]

company = pd.DataFrame([])
company = company.append(p1_company)
company = company.append(p2_company)
company = company.append(p3_company)
company = company.append(p4_company)
company = company.append(p5_company)
company = company.drop_duplicates('COMPANY_ID')

# company.to_pickle('company.pick')
print(company.shape)

(908520, 5)


In [ ]:
# COMPANY NAME, ADDR 정제
def cleanChar(sentence):
    import re
    pattern = re.compile(r'\s{0,2}\&amp\;\s{0,2}')
    sentence = re.sub(pattern,'@',sentence)
    pattern = re.compile(r'\s{0,2}\&.{2,4}\;\s{0,2}|\<|\>|\'|\"')
    sentence = re.sub(pattern,'',sentence)
    pattern = re.compile(r'\&')
    sentence = re.sub(pattern,'@',sentence)
    return sentence

company[['ADDR','CO_NAME']]=company[['ADDR','CO_NAME']].applymap(cleanChar)

print(company.shape)

In [ ]:
def refineHana(data):
    print('total: ', data.shape)
    
    #화장실제거
    test0 = data[~data['CO_NAME'].str.contains('화장실')]
    print('nonRestroom: ',test0.shape)
    test = data[data['CO_NAME'].str.contains('화장실')]  
    print('Restroom: ',test.shape)
    
    #1
    test_public = test[test['CATE']=='공공']  
    
    print('Restroom-public: ',test_public.shape)    
    etc = test_public[~test_public['CO_NAME'].str.contains('주유소|충전소|가스|병원|수련관|복지관')]
    a = test_public[test_public['CO_NAME'].str.contains('주유소')].assign(CATE = '교통', CATE1 = '주유소')
    b = test_public[test_public['CO_NAME'].str.contains('충전소|가스')].assign(CATE = '교통', CATE1 = '충전소')
    c = test_public[test_public['CO_NAME'].str.contains('병원')].assign(CATE = '의료/건강', CATE1 = '병원')
    d = test_public[test_public['CO_NAME'].str.contains('수련관')].assign(CATE = '교육', CATE1 = '수련원')
    e = test_public[test_public['CO_NAME'].str.contains('복지관')].assign(CATE = '공공', CATE1 = '사회복지시설')
    test_public = etc.append([a,b,c,d,e])
    
    print('after Restroom-public: ',test_public.shape)    
    test_public=test_public.assign(CO_NAME=test_public['CO_NAME'].str.replace('공중화장실','')) #460
    test_public=test_public.assign(CO_NAME=test_public['CO_NAME'].str.replace('개방화장실',''))#1157
    test_public=test_public.assign(CO_NAME=test_public['CO_NAME'].str.replace('화장실',''))#20 - 충전소 화장실, 협동조합 화장실
    # A=A.assign(CO_NAME=A['CO_NAME'].str.replace('공중화장실',''))
    
    #2
    test_nonPublic = test[test['CATE']!='공공']
    print('Restroom-nonPublic: ',test_nonPublic.shape)
    test_nonPublic=test_nonPublic.assign(CO_NAME=test_nonPublic['CO_NAME'].str.replace('공중화장실','')) #460
    test_nonPublic=test_nonPublic.assign(CO_NAME=test_nonPublic['CO_NAME'].str.replace('개방화장실',''))#1157
    test_nonPublic=test_nonPublic.assign(CO_NAME=test_nonPublic['CO_NAME'].str.replace('화장실',''))#20 - 충전소 화장실, 협동조합 화장실
    # test_nonPublic.CO_NAME=test_nonPublic['CO_NAME'].str.replace('공중화장실','') #460
    print('after Restroom-nonPublic: ',test_nonPublic.shape)
    
    #3
    temp = test_public.append(test_nonPublic)
    print('check Restroom: ',temp.shape)
    
#     temp.append(test0).sort_values(by=['EPOPCON_ID','COMPANY_ID','CARD_NAME'], axis=0)
    print('check total: ',temp.append(test0).shape)
    return  temp.append(test0)

company = refineHana(company)
print(company.shape)
company.to_pickle('company.pick')

### MERGE DATA + CLEAN(CARD_NAME/COMPANY_NAME/ADDR)

In [13]:
p = pd.DataFrame([])
p = p.append(p1)
p = p.append(p2)
p = p.append(p3)
p = p.append(p4)
p = p.append(p5)
print(p.shape) 

#1.  CARD_NAME 정제 
#a.find('(')
def cleanData(sentence):
    import re
    pattern = re.compile(r'\(.{0,4}\)|RE\)|\<현장즉시결제\>|\[|\s|카드|\<|\>|\'|\"')
    sentence = re.sub(pattern,'',sentence)
    return sentence # return 안쓰면 none값 뜬다.
    
p = p.assign( CARD_NAME = p['CARD_NAME'].apply(cleanData) )
p.to_pickle('cardname_clean.pick')
p.head()

(18326539, 4)


,CARD_NAME,COMPANY_ID,EPOPCON_ID,PAYMENT
0,삼성가족,918472,cjcis15a392e136259ad0a9e3287e5,130711
1,하나카드,7905275,cjcis15a7c97905a93f40fafc73980,2800
2,신한체크,7241106,cjcis15a39c74e5a93f40965bde60d,3000
3,삼성카드,6950135,cjcis15a7ec6cab2675635e2581185,300000
4,현대카드,5765476,cjcis15a751f3fe946d9036136c359,109000


In [2]:
company= pd.read_pickle('company.pick')
lifezone= pd.read_pickle('lifezone.pick')
p= pd.read_pickle('cardname_clean.pick')
p.head(2)

,CARD_NAME,COMPANY_ID,EPOPCON_ID,PAYMENT
0,삼성가족,918472,cjcis15a392e136259ad0a9e3287e5,130711
1,하나,7905275,cjcis15a7c97905a93f40fafc73980,2800
2,신한체크,7241106,cjcis15a39c74e5a93f40965bde60d,3000
3,삼성,6950135,cjcis15a7ec6cab2675635e2581185,300000
4,현대,5765476,cjcis15a751f3fe946d9036136c359,109000


### payment type 변환

In [3]:
p['PAYMENT']= p['PAYMENT'].astype(float)
p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18326539 entries, 0 to 4173991
Data columns (total 4 columns):
CARD_NAME     object
COMPANY_ID    int64
EPOPCON_ID    object
PAYMENT       float64
dtypes: float64(1), int64(1), object(2)
memory usage: 699.1+ MB


### SUM/GROUP_BY + RENAME_COLUMNS

 - 전체 column 기준 groupby
 - p_group = p.groupby(['EPOPCON_ID','COMPANY_ID','CARD_NAME']).agg({'CNT_PAY':'sum','SUM_PAY':'sum'}).reset_index()
 - http://pinkwink.kr/977

In [4]:
# df = data.groupby('Seed').agg(
#     {'age':['sum'],
#      'height':['mean', 'std']})
# p_group.columns = ["_".join(x) for x in p_group.columns.ravel()]

p_group = p.groupby(['EPOPCON_ID','COMPANY_ID','CARD_NAME']).agg(['count','sum']).reset_index()
p_group.columns = ["_".join(x) for x in p_group.columns.ravel()]

print(p_group.shape) # (12266736, 5) -> 12265619 감소 (카드이름으로 더 많이 묶임)
p_group.head()

(12265619, 5)


,EPOPCON_ID_,COMPANY_ID_,CARD_NAME_,PAYMENT_count,PAYMENT_sum
0,cjcis15a0e888b71e42719f785db8b,27879,기업BC,2,12000.0
1,cjcis15a0e888b71e42719f785db8b,46177,기업BC,2,99300.0
2,cjcis15a0e888b71e42719f785db8b,207811,기업BC,1,2000.0
3,cjcis15a0e888b71e42719f785db8b,255730,기업BC,1,6000.0
4,cjcis15a0e888b71e42719f785db8b,331119,기업BC,1,7500.0


In [5]:
del p
New_Labels = ['EPOPCON_ID','COMPANY_ID','CARD_NAME','CNT_PAY','SUM_PAY'] 
p_group.columns = New_Labels
p_group.head()


,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY
0,cjcis15a0e888b71e42719f785db8b,27879,기업BC,2,12000.0
1,cjcis15a0e888b71e42719f785db8b,46177,기업BC,2,99300.0
2,cjcis15a0e888b71e42719f785db8b,207811,기업BC,1,2000.0
3,cjcis15a0e888b71e42719f785db8b,255730,기업BC,1,6000.0
4,cjcis15a0e888b71e42719f785db8b,331119,기업BC,1,7500.0


### JOIN

In [6]:
p = pd.merge(p_group,company, how='inner', on='COMPANY_ID')
p = pd.merge(p,lifezone, how='inner', on='EPOPCON_ID')

# EPOPCON_ID 정제
# p = p.assign( EPOPCON_ID = p['EPOPCON_ID'].apply(lambda x : x.replace('cjcis','')) )
print(p.shape)
p.head()

(9961401, 11)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
0,cjcis15a0e888b71e42719f785db8b,27879,기업BC,2,12000.0,경기도 수원시 장안구 송죽동 453-29번지,마트,편의점,CU 수원경기일보점,37.4960912,127.0183798
1,cjcis15a0e888b71e42719f785db8b,46177,기업BC,2,99300.0,경기 성남시 분당구 정자동 178-1 NAVER 그린팩토리 1층,온라인쇼핑,온라인쇼핑,네이버,37.4960912,127.0183798
2,cjcis15a0e888b71e42719f785db8b,207811,기업BC,1,2000.0,서울 서초구 서초동 1690-1 삼풍프라자 지하1층,생활,생활용품,다이소 교대역점,37.4960912,127.0183798
3,cjcis15a0e888b71e42719f785db8b,255730,기업BC,1,6000.0,서울 서초구 서초동 1669-6 동진빌딩 1층,외식,한식,돌배기집 교대역점,37.4960912,127.0183798
4,cjcis15a0e888b71e42719f785db8b,331119,기업BC,1,7500.0,서울 서초구 서초동 1674-5,외식,패스트푸드,맥도날드 서울교대점,37.4960912,127.0183798


### epopcon_id에서 cjcis 문자제거하기

In [7]:
# EPOPCON_ID 정제
p = p.assign( EPOPCON_ID = p['EPOPCON_ID'].apply(lambda x : x.replace('cjcis','')) )
print(p.shape)
p.head()

(9961401, 11)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
0,15a0e888b71e42719f785db8b,27879,기업BC,2,12000.0,경기도 수원시 장안구 송죽동 453-29번지,마트,편의점,CU 수원경기일보점,37.4960912,127.0183798
1,15a0e888b71e42719f785db8b,46177,기업BC,2,99300.0,경기 성남시 분당구 정자동 178-1 NAVER 그린팩토리 1층,온라인쇼핑,온라인쇼핑,네이버,37.4960912,127.0183798
2,15a0e888b71e42719f785db8b,207811,기업BC,1,2000.0,서울 서초구 서초동 1690-1 삼풍프라자 지하1층,생활,생활용품,다이소 교대역점,37.4960912,127.0183798
3,15a0e888b71e42719f785db8b,255730,기업BC,1,6000.0,서울 서초구 서초동 1669-6 동진빌딩 1층,외식,한식,돌배기집 교대역점,37.4960912,127.0183798
4,15a0e888b71e42719f785db8b,331119,기업BC,1,7500.0,서울 서초구 서초동 1674-5,외식,패스트푸드,맥도날드 서울교대점,37.4960912,127.0183798


In [8]:
p.to_pickle('hana_march.pick')

### EXPORT XML FILE 

- UnicodeDecode Error : illegal multibyte sequence

p=p.replace('\\xa0', '')  ->  with open(filename, mode, encoding='utf8') as f:

In [15]:
#https://stackoverflow.com/questions/18574108/how-do-convert-a-pandas-dataframe-to-xml

def to_xml(df, filename=None, mode='w'):
    bigxml=['<?xml version="1.0"?>','<resultset xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">']
    def row_to_xml(row):
        xml = [' <row>']
        for i, col_name in enumerate(row.index):
            xml.append('  <field name="{0}">{1}</field>'.format(col_name, row.iloc[i]))
        xml.append(' </row>')
        return '\n'.join(xml)
    resultset = '\n'.join(df.apply(row_to_xml, axis=1))
    bigxml.append(resultset)
    bigxml.append('</resultset>')
    res = '\n'.join(bigxml)
    
    if filename is None:
        return res
    with open(filename, mode, encoding='utf8') as f:
        f.write(res)

pd.DataFrame.to_xml = to_xml

#tmp.to_xml('test.csv')
p.iloc[:2500000,].to_xml('hana_march1.xml')
p.iloc[2500000:5000000,].to_xml('hana_march2.xml')
p.iloc[5000000:7500000,].to_xml('hana_march3.xml')
p.iloc[7500000:9962239,].to_xml('hana_march4.xml')

In [10]:
p.iloc[1006946:,].head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
1006946,15a3991ec71e42756fbe86b6e,30188,KB국민,1,9200.0,경기도 수원시 장안구 정자동 535-12,마트,편의점,CU 청솔번영점,37.2985548,126.9872756
1006947,15a3991ec71e42756fbe86b6e,8077641,KB국민체크,1,7400.0,경기 수원시 장안구 천천동 527-1,마트,시장,장독대천천동점,37.2985548,126.9872756
1006948,15a3991ec71e42756fbe86b6e,7486252,KB국민,1,5100.0,경기도 수원시 장안구 정자동 877-4 정연메이져 1층,카페/디저트,베이커리,뚜레쥬르수원정자점,37.2985548,126.9872756
1006949,15a3991ec71e42756fbe86b6e,5256211,KB국민,1,2200.0,,뷰티,화장품,올리브영 수원천천점,37.2985548,126.9872756
1006950,15a3991ec71e42756fbe86b6e,5256211,KB국민체크,1,6210.0,,뷰티,화장품,올리브영 수원천천점,37.2985548,126.9872756


In [22]:
p[p['CO_NAME'].str.contains('\&|\<|\>|\'|\"')].shape
p[p['CO_NAME'].str.contains('화장실')].shape

(0, 11)

In [21]:
p[p['ADDR'].str.contains('&|<|>|\'|"')].shape
p[p['ADDR'].str.contains('@')].shape
p[p['ADDR'].str.contains('@')].head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
1528,15a38a9795a93f40965bb6f22,7650379,KB국민,1,20000.0,서울 마포구 상암동 1605 누리꿈스퀘어 R@D 타워 9062,통신,인터넷,뮤즈플랫폼,37.5797684,126.8902617
1568,15a38a98fb267560d56e2ccc4,917117,KB국민,1,167900.0,서울 마포구 상암동 1606 CJ E@M센터 1층,카페/디저트,카페,주스솔루션 상암E@M점,37.5667030,126.9783130
1569,15a38a98fb267560d56e2ccc4,946797,KB국민,1,9200.0,서울 마포구 상암동 1606 상암DMC CJ E@M 센터 1층,카페/디저트,카페,투썸플레이스 상암E@M점,37.5667030,126.9783130
2635,15a38ac425a93f40965bb6f95,534893,신한,2,410000.0,서울특별시 도봉구 창동 27-3 주공@19단지202;203;208호,교육,,솔빛보습학원,37.6557412,127.0543807
5200,15a38afe25a93f40965bb70a0,5004828,KB국민,2,12700.0,경기 용인시 기흥구 구갈동 581-1 강남@플러스 1층,카페/디저트,카페,스타벅스 강남대점,37.5667030,126.9783130


In [17]:
p[p['CARD_NAME'].str.contains('\&|\<|\>|\'|"')].shape

(0, 11)